# Let a Cart Climb Up a Hill

### Problem Formulation

MDP: <br>
Let state X = (position, velocity), where position $\in$ [-1.2,0.6], and velocity $\in$ [-0.7, 0.7]. By discretizing the state, we get state X as a NxN matrix (N is the density)(N = 30 by default).
<br>
Let control U be a scalar number $\in [0,1,2]$. <br>
The transition/motion model $p(x'|x,u)$ is unknown. <br>
The stage reward (negative of stage cost), $l(x,u) = r$, is also unknown. <br>
The terminal cost is also unkown.<br> <br>
SARSA & Q-learning off-policy TD: <br>
The Q function is a NxNx3 matrix, where each entry Q(x,u) cooresponds to state & action pair (x,u). <br>
The policy P is a NxNx3 matrix, where each entry P(x,u) cooresponds to each entry in Q. <br>
Also, we also need some additional hyperparameters: $\epsilon, \gamma, \alpha$, where $\epsilon$ is $\in (0,1)$ used in epsilon-greedy policy, and decreasing. $\gamma$ is the discount factor which is $\in (0,1)$. $\alpha$ is the learning rate.   

### Technical Approach 

greey policy: $(1-\epsilon)$ probability choose according to policy. $\epsilon$ probability choose a random action. <br>
    
SARSA: <br>
initialize Q <br>
loop until convergence (total Q updates is less than threshold): <br>
&emsp; t = 0 <br>
&emsp;  given initial state x <br>
&emsp;  pi <-- greedy policy from Q <br>
&emsp;   while not done or t<200: <br>
&emsp;&emsp; u = pi(x)<br>
&emsp;&emsp; x',r' = env.step(u)<br>
&emsp;&emsp; u' = pi(s')<br>
&emsp;&emsp; $Q(x,u) = Q(x,u) + \alpha*(r' + \gamma*(Q(x',u')-Q(x,u)))$<br>
&emsp;&emsp; x = x'<br>
&emsp;&emsp; t = t+1<br>
<br>
Off-policy TD with Q-learning: <br>
initialize Q <br>
loop until convergence (total Q updates is less than threshold): <br>
&emsp; t = 0 <br>
&emsp;  given initial state x <br>
&emsp;  pi <-- greedy policy from Q <br>
&emsp;   while not done or t<200: <br>
&emsp;&emsp; u = pi(x)<br>
&emsp;&emsp; x',r' = env.step(u)<br>
&emsp;&emsp; u' = pi(s')<br>
&emsp;&emsp; $Q(x,u) = Q(x,u) + \alpha*(r' + \gamma*(max_{u'}[Q(x',u')]-Q(x,u)))$<br>
&emsp;&emsp; x = x'<br>
&emsp;&emsp; t = t+1<br>
<br>
Notice that if x is the terminal state (env.step return done before t reaches 200), then, treat Q(x',u') as 0. <br>

Conditions for convergence: <br>
As loops goes to infinite time: <br>
&emsp; epsilon-greedy needs to be GLIE: 1. all state-control pairs are explored infinitely. 2. epsilon-greedy policy converges to a greedy policy. <br>
&emsp; learning rate, $\alpha$, is Robbins-Monro. <br>
<br>
My choice of hyperparameters: <br>
intial $\epsilon_i$ = 100. And in each loop t, $\epsilon_t = \epsilon_i/t$. N(density) = 30. Discount factor $\gamma$ = 0.9. And according to Piazza Post @219, I decide to choose a fixed small $\alpha = 0.1$ as the learning rate. 

### Result


Choice of hyper-parameters is at the last section in technical approach. The final optimal policy of off-policy and on-policy are similar. After running several test trails with their optimal policy, I found that although the car reaches the goal before 200 steps, the total steps it took are unstable due to the random start state. The running time of off-policy is a little slow due to the additional max function in update formula.  By comparing the V(x) plots below, we can see that the plot of value function at start state (0,0) of on-policy and off-policy are very similar. It is what expected, since start state is visited same amount of times for both on and off policy. In off-policy plot, there is a big jump at state 2 and state 3, since off-policy has max function in update and the positions are seldom visited. <br>
Also, all plots does not converges at the end of 10000th episode, since the learning is probably too small or the density is too high.  


# Problem 4

### Problem Formulation

MDP: <br>
Let state X be pair of $(x_1,x_2)$ which is (angle, velocity). The angle is $\in [0,2\pi]$. The velocity is $\in [-Vmax, Vmax]$. By discretizing the state, we get state X as N1xN2 matrix, where N1 is the density of angle, and N2 is the density of the velocity. (N1 = 100, N2 = 50, Vmax = 3 by default) <br>
Let control U $\in [-Umax, Umax]$. By discretizing the U, we get U as a N3-length array, where N3 is the density of the control. (Umax = 1, N3 = 30, by default) <br>
Let stage cost be $l(x,u)dt = (1-exp(kcosx_1-k)+\frac{r}{2}u^2)dt$ <br>
Let motion model $p_f(x'|x,u) \sim N(x+f(x,u)dt,\sigma\sigma^Tdt)$, where $f(x,u) = [x_2,a*sinx_1-b*x_2+u]$. (a=1, b=0.8, dt=1, k=100, r=1, $\sigma$ = 0.1 by default) <br>

Optimization problem: <br>
The value function V is a N1xN2 matrix where each entry cooresponding to optimal value of each state. <br>
The policy P is a N1xN2 matrix where each entry cooresponding to optimal control of each state. <br>
Also, for the discount factor $\gamma$ is set equal to 0.9 as default. <br>
<br>
Interpolation problem: <br>
Let t be an array that stores discrete time.
Let x1 be an array that stores the x-axis positions over the time.
Let x2 be an array that stores the y-axis positions over the time.
Let theta be an array that stores angles over the time.
Let u be an array that stores the controls over the time.

### Technical Approach

Initalize motion model $p_f$, which is a matrix with dimensionN1xN2xN3xN1xN2: <br>
For each state-action pair(x1,x2,u): 1. Calculate out $x+f(x,u)dt$, which is the mean of $p_f(x'|x1,x2,u)$. 2. for each state, calculate out its distance to the mu. Notice that the angle 0 to 2*pi is circular, thus the distance between two angles should be $min(|\theta1-\theta2|, 2\pi-|\theta1-\theta2|)$. 3. Assign guassian probability ($\sigma\sigma^Tdt$ covariance) to each states according to how far they are actually away from the mean. 4. Normalize the probabilies of all states so that they sum up to one. <br>
<br>
Value Iteration: <br>
Intialize $p_f$ (Set $p_f(x'|x,u) \sim N(x+f(x,u)\delta t, \sigma \sigma^T \delta t)$) <br>
Initialize V for each state <br>
loop until convergence: <br>
&emsp; for all x in X:<br>
&emsp;&emsp; $V^{t+1}(x) = min_u(l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V^t(x')])$<br>
<br>
Policy Iteration: <br>
Intialize $p_f$ (Set $p_f(x'|x,u) \sim N(x+f(x,u)\delta t, \sigma \sigma^T \delta t)$) <br>
Initialize Policy for each state <br>
loop until convergence: <br>
&emsp; for all x in X:<br>
&emsp;&emsp; u <-- Current Policy, $P^t$ <br>
&emsp;&emsp; Get linear equation: $V(x) = l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V(x')])$<br>
&emsp; Solve the above linear equations as a big linear system (Use python np.linalg.solve(a,b))<br>
&emsp; Then you get V for all states <br>
&emsp; for all x in X: <br>
&emsp;&emsp; $P^{t+1}(x) = argmin_u(l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V(x')])$ <br>
<br>
Interpolation: <br>
Following the code in PendulumProblem.py, all we need to do is: <br>
Intialize theta_i, and v_i<br>
Let $self.theta = [], self.u = []$ <br>
for i in range(self.t):<br>
&emsp; Get u from policy by mapping theta_i,v_i to closet pair in discretized states <br>
&emsp; Update theta_i, v_i by randomly generate (angle, velocity) pair from motion model $p_f$ <br>
&emsp; self.theta.append(theta_i), self.u.append(u) <br>
Then, self.x1 = np.sin(self.theta), self.y1 = np.cos(self.theta). <br>
Then, just run the code in PendulumProblem.py. <br>